# 17c - HQSL v2 calculation (express)

This notebook does the same as notebook 17b but allows iterating over polygons.

## Import libraries

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm 

import os
import sys
module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

In [2]:
import h3
def neighbour_mean(hex_id, hex_id_name, hex_bins, col_name):
    return hex_bins.loc[hex_bins[hex_id_name].isin(h3.k_ring(hex_id,1)),col_name].mean()

## Notebook config

In [3]:
# ---------- AREAS OF ANALYSIS TO BE PROCESSED
# allows for ['santiago','zonascensales','comunas','unidadesvecinales','alameda','hex']
areas_of_analysis_list = ['santiago','comunas','unidadesvecinales','alameda']

# ---------- RESOLUTION OF HEX (IF NEEDED)
# If area_analysis = 'hex', ** NEED ** to input hexs resolution
res = 10

# ---------- PERSONA SPEED
walk_speed = 4.5
speed_name = str(walk_speed).replace('.','_')

# ---------- LOCAL DIRECTORIES
# 'alex' or 'edgar'
all_local_dirs = 'alex'

if all_local_dirs == 'alex':
    grl_dir = "../../../data/external/temporal_todocker/santiago/proximidad/"        
elif all_local_dirs == 'edgar':
    grl_dir = "../../../data/processed/santiago/"
else:
    print("WARNING: Fix all_local_dirs variable")

# ---------- SAVING
# save variables (preprocessed) output to database?
upload_preprocessed = True
# save FINAL (index) output to database?
save = True
# how to save to database?
if_exists='replace'
# save to local?
save_local = False

# NOTE THAT LOCAL DATA USE IS INTEGRATED IN FUNCTION (True if area_analysis == 'alameda' or 'hexagon')

## Base data required - Parameters and weights dictionaries

In [4]:
# ---------- PARAMETERS AND WEIGHT DICTS
# Structure: {social_functions:{themes:[source_names]}}
parameters_dict = {'supplying':{'wellbeing':['carniceria', #Accessibility to Butcher/Fish Shops
                                             'hogar', #Accessibility to Hardware/Paint Shops
                                             #Not available: Accessibility to Greengrocers
                                             'bakeries', #Accessibility to Bakeries and delis
                                             'supermercado',#Accessibility to supermarkets
                                             'banco'#Accessibility to bank
                                            ],
                                'sociability':['ferias',#Accessibility to city fairs/markets
                                               'local_mini_market',#Accessibility to local and mini markets
                                               'correos'#ADDED: MAIL SERVICE
                                              ],
                                'environmental_impact':['centro_recyc'#Accessibility to recycling center
                                                        #Not available: Accessibility to compost
                                                       ]
                               },
                   'caring':{'wellbeing':['hospital', #Accessibility to hospital
                                          'clinica',#Accessibility to public clinics
                                          'farmacia',#Accessibility to pharmacies
                                          'vacunatorio',#Accessibility to vaccination center
                                          'consult_ado',#Accessibility to optician/audiologist(###ADDED DENTIST)
                                          'salud_mental',###ADDED: MENTAL HEALTH
                                          'labs_priv',###ADDED: LABORATORIES
                                          'residencia_adumayor'###ADDED: ELDERLY PERMANENT RESIDENCIES
                                         ],
                             'sociability':['eq_deportivo',#Accessibility to sports equipments
                                            'club_deportivo'#Accessibility to sport clubs
                                           ],
                             'environmental_impact':['noise',
                                                     'temp'
                                 #Not available: Air polution
                                                    ]
                            },
                   'living':{'wellbeing':['civic_office',#Accessibility to civic offices
                                          #Not available: Number of street bentches
                                          'tax_collection',#ADDED: AFIP(TAX COLLECTOR)
                                          'social_security',#ADDED: SOCIAL SECURITY
                                          'police',#Accessibility to police(###MOVED FROM LIVING TO CARING)
                                          'bomberos'#Accessibility to fire stations
                                          #Not available: Accessibility to street lamp
                                         ],
                             'sociability':['houses',#Accessibility to permanent residencies
                                            'social_viv',#Accessibility to social housing
                                            #Not available: Accessibility to student housing
                                            'hotel'#ADDED: HOTELS
                                           ],
                             'environmental_impact':['inter',
                                                     #Not available: Corrected compactness
                                                     #Not available: Width of sidewalks
                                                    ],
                            },
                   'enjoying':{'wellbeing':['museos',#Accessibility to museums
                                            #Not available: Accessibility to theater,operas
                                            'cines',#Accessibility to cinemas
                                            'sitios_historicos',#Accessibility to historical places
                                            'ndvi'#Number of trees
                                           ],
                               'sociability':['restaurantes_bar_cafe',#Accessibility to bars/cafes + Accessibility to restaurants
                                              'librerias',#Accessibility to record and book stores, galleries, fairs
                                              #Not available: Accessibility to cultural and/or formative spaces
                                              #Not available: Accessibility to places of workship
                                              'ep_plaza_small'#Accessibility to boulevards, linear parks, small squares + Accessibility to squares
                                             ],
                               'environmental_impact':['ep_plaza_big'#Accessibility to big parks
                                                       #Not available: Accessibility to shared gardens
                                                       #Not available: Accessibility to urban playgrounds
                                                      ]
                              },
                   'learning':{'wellbeing':['edu_basica_pub',#'edu_basica_priv',#Accessibility to public elementary school
                                            'edu_media_pub',#'edu_media_priv',#Accessibility to public high school
                                            'jardin_inf_pub',#'jardin_inf_priv',#Similar to Accessibility to childcare
                                            'universidad',#Accessibility to university
                                            'edu_tecnica',#ADDED: TECHNICAL EDUCATION
                                           ],
                               'sociability':['edu_adultos_pub',#'edu_adultos_priv',#Accessibility to adult formation centers
                                              'edu_especial_pub',#'edu_especial_priv',#Accessibility to specialized educational centers
                                              #Not available: Accesibility to establishments and services for disabled adults
                                              'bibliotecas'#Accessibility to libraries(###MOVED FROM ENJOYING TO LEARNING)
                                             ],
                               'environmental_impact':['centro_edu_amb'#Accessibility to centers for learning environmental activities
                                                       #Not available: Accessibility to gardening schools
                                                      ],
                              },
                   'working':{'wellbeing':['paradas_tp_ruta',#Accessibility to bus stop
                                           'paradas_tp_metro',#Accessibility to metro
                                           'paradas_tp_tren'#Accessibility to train stop
                                          ],
                              'sociability':['oficinas'#Accessibility to office
                                             #Not available: Accessibility to incubators
                                             #Not available: AccSeveral other articles cite 60dB as a safe noise zone. essibility to coworking places
                                          ],
                              'environmental_impact':['ciclovias',
                                                      'estaciones_bicicletas'#Accessibility to bike lanes
                                                      #Not available: Accessibility to shared bike stations
                                                     ]
                             }
                  }

weight_dict = {'carniceria':'rare', #SUPPLYING
               'hogar':'rare',
               'bakeries':'rare',
               'supermercado':'rare',
               'banco':'rare',
               'ferias':'rare',
               'local_mini_market':'rare',
               'correos':'very_rare',
               'centro_recyc':'rare',
               #CARING
               'hospital':'very_rare',
               'clinica':'rare',
               'farmacia':'rare',
               'vacunatorio':'very_rare',
               'consult_ado':'very_rare',
               'salud_mental':'very_rare',
               'labs_priv':'very_rare',
               'residencia_adumayor':'rare',
               'eq_deportivo':'rare',
               'club_deportivo':'rare',
               'noise':'specific',
               'temp':'specific',
               #LIVING
               'civic_office':'rare', 
               'tax_collection':'very_rare',
               'social_security':'very_rare',
               'police':'very_rare',
               'bomberos':'very_rare',
               'houses':'specific',
               'social_viv':'specific',
               'hotel':'rare',
               'inter':'specific',
               #ENJOYING
               'museos':'very_rare',
               'cines':'very_rare',
               'sitios_historicos':'rare',
               'ndvi':'specific',
               'restaurantes_bar_cafe':'frequent',
               'librerias':'rare',
               'ep_plaza_small':'frequent',
               'ep_plaza_big':'rare',
               #LEARNING
               'edu_basica_pub':'rare', 
               'edu_media_pub':'rare',
               'jardin_inf_pub':'rare',
               'universidad':'very_rare',
               'edu_tecnica':'very_rare',
               'edu_adultos_pub':'rare',
               'edu_especial_pub':'rare',
               'bibliotecas':'very_rare',
               'centro_edu_amb':'very_rare',
               #WORKING
               'paradas_tp_ruta':'frequent',
               'paradas_tp_metro':'very_rare',
               'paradas_tp_tren':'very_rare',
               'oficinas':'specific',
               'ciclovias':'rare',
               'estaciones_bicicletas':'rare',
              }

## Base data required - Scale function definitions

In [5]:
def rare_fn(cont):
    if cont == 0:
        res_val = 0
    elif cont > 0 and cont < 2:
        res_val = res_val_regression(0, 2, 0, 2.5, cont)
    elif cont >= 2 and cont < 4:
        res_val = res_val_regression(2, 4, 2.5, 5, cont)
    elif cont >= 4 and cont < 7:
        res_val = res_val_regression(4, 7, 5, 7.5, cont)
    elif cont >= 7 and cont < 10:
        res_val = res_val_regression(7, 10, 7.5, 10, cont)
    elif cont >= 10:
        res_val = 10
    
    return res_val


def very_rare_fn(cont):
    min_x = 0
    max_x = 1
    min_y = 0
    max_y = 10
    
    return res_val_regression(min_x, max_x, min_y, max_y, cont)


def frequent_fn(cont):
    if cont == 0:
        res_val = 0
    elif cont > 0 and cont < 6:
        res_val = res_val_regression(0, 6, 0, 2.5, cont)
    elif cont >= 6 and cont < 12:
        res_val = res_val_regression(6, 12, 2.5, 5, cont)
    elif cont >= 12 and cont < 18:
        res_val = res_val_regression(12, 18, 5, 7.5, cont)
    elif cont >= 18 and cont < 25:
        res_val = res_val_regression(18, 25, 7.5, 10, cont)
    elif cont >= 25:
        res_val = 10
    
    return res_val


def res_val_regression(min_x, max_x, min_y, max_y, cont):
    slope = (max_y-min_y)/(max_x-min_x)
    intersect = min_y - slope * min_x
    res_val = slope * cont + intersect
    if cont > max_x:
        res_val = max_y
        
    return res_val


def office_fn(cont):
    if cont == 0:
        res_val = 0
    elif cont > 0 and cont < 2.823938308:
        res_val = res_val_regression(0, 2.823938308, 0, 2.5, cont)
    elif cont >= 2.823938308 and cont <  5.539263604:
        res_val = res_val_regression(2.823938308, 5.539263604, 2.5, 5, cont)
    elif cont >= 5.539263604 and cont < 10.96991420:
        res_val = res_val_regression(5.539263604, 10.96991420, 5, 7.5, cont)
    elif cont >= 10.96991420 and cont < 16.40056479:
        res_val = res_val_regression(10.96991420, 16.40056479, 7.5, 10, cont)
    elif cont >= 16.40056479:
        res_val = 10
    
    return res_val


def ndvi_fn(cont):
    min_x = 0
    max_x = 0.4
    min_y = 0
    max_y = 10
    if cont > max_x:
        return 10
    elif cont <= min_x:
        return 0
    else:
        return res_val_regression(min_x, max_x, min_y, max_y, cont)


def inter_fn(cont):
    min_x = 20
    max_x = 100
    min_y = 0
    max_y = 10
    if cont > max_x:
        return 10
    elif cont < min_x:
        return 0
    else:
        return res_val_regression(min_x, max_x, min_y, max_y, cont)


def noise_fn(cont):
    min_x = 55
    max_x = 70
    min_y = 10
    max_y = 0
    if cont > max_x:
        return 0
    elif cont < min_x:
        return 10
    else:
        return res_val_regression(min_x, max_x, min_y, max_y, cont)


def temp_fn(cont, mean, std):
    if cont >= (mean + 2*std):
        res_val = 0
    elif cont < (mean + 2*std) and cont >= (mean + std):
        res_val = res_val_regression((mean + std), (mean + 2*std), 2.5, 0, cont)
    elif cont < (mean + std) and cont >= (mean):
        res_val = res_val_regression((mean), (mean + std), 5, 2.5, cont)
    elif cont < (mean) and cont >= (mean - std):
        res_val = res_val_regression((mean - std), (mean), 7.5, 5, cont)
    elif cont < (mean - std) and cont >= (mean - 2*std):
        res_val = res_val_regression((mean - 2*std), (mean - std), 10, 7.5, cont)
    elif cont < (mean - 2*std):
        res_val = 10
    if area_analysis == 'santiago':
        res_val = 5
    
    return res_val


def household_fn(cont):
    res_val = res_val_regression(0, 50, 0, 10, cont)
    
    return res_val

    
def social_viv_fn(cont):
    min_x = 0
    max_x = 20
    min_y = 0
    max_y = 10
    if cont > max_x:
        return 10
    elif cont < min_x:
        return 0
    else:
        return res_val_regression(min_x, max_x, min_y, max_y, cont)


def specific_fn(cont, source, mean, std):
    if 'ndvi' in source:
        return ndvi_fn(cont)
    elif 'inter' in source:
        return inter_fn(cont)
    elif 'noise' in source:
        return noise_fn(cont)
    elif 'temp' in source:
        return temp_fn(cont, mean, std)
    elif 'houses' in source:
        return household_fn(cont)
    elif 'social_viv' in source:
        return social_viv_fn(cont)
    elif 'oficinas' in source:
        return office_fn(cont)


def scale_source_fn(cont, source, weight_dict, mean, std):
    if weight_dict[source] == 'rare':
        return rare_fn(cont)
    elif weight_dict[source] == 'very_rare':
        return very_rare_fn(cont)
    elif weight_dict[source] == 'frequent':
        return frequent_fn(cont)
    elif weight_dict[source] == 'specific':
        return specific_fn(cont, source, mean, std)

## Base data required - Index function definitions

In [6]:
def hqsl_fn(hex_gdf, parameters_dict):

    hex_gdf = hex_gdf.copy()
    
    social_function_list = []
    
    for social_function in parameters_dict.keys():
        social_function_list.append(social_function)
    
    hex_gdf['hqsl'] = hex_gdf[social_function_list].sum(axis=1)

    base_columns = [code_column,'geometry']
    filter_list = ['hqsl']
    filter_list.extend(base_columns)
    hex_gdf = hex_gdf[filter_list].copy()
    
    return hex_gdf


def social_fn(hex_gdf, parameters_dict):
    
    hex_gdf = hex_gdf.copy()
    
    for social_function in parameters_dict.keys():
        source_list = []
        
        for indicator in parameters_dict[social_function].keys():
            source_list.extend(parameters_dict[social_function][indicator])
        
        source_list = [s+'_scaled' for s in source_list]
        hex_gdf[social_function] = hex_gdf[source_list].mean(axis=1)

    base_columns = [code_column,'geometry']
    filter_list = list(parameters_dict.keys())
    filter_list.extend(base_columns)
    hex_gdf = hex_gdf[filter_list].copy()
    
    return hex_gdf


def indicator_fn(hex_gdf, parameters_dict):
    hex_ind = hex_analysis.copy()

    filter_list = []
    
    indicator_list = list(set().union(*parameters_dict.values()))
    for indicator in indicator_list:
        social_indicator = []
        
        for social_function in parameters_dict.keys():
            social_indicator.append(indicator+'_'+social_function)
            
            source_indicator = parameters_dict[social_function][indicator]
            source_indicator = [s+'_scaled' for s in source_indicator]
            
            hex_ind[indicator+'_'+social_function] = hex_ind[source_indicator].mean(axis=1)
    
        hex_ind[indicator] = hex_ind[social_indicator].sum(axis=1)
        filter_list.extend(social_indicator)
        filter_list.append(indicator)
    
    base_columns = [code_column,'geometry']
    filter_list.extend(base_columns)
    hex_ind = hex_ind[filter_list].copy()
            
    return hex_ind

## Run

In [9]:
for area_analysis in areas_of_analysis_list:
    print("--"*40)
    print(f"--- RUNNING CALCULATION FOR AREA OF ANALYSIS: {area_analysis}.")

    ##########################################################################################
    # Define use of local data
    if (area_analysis == 'alameda') or (area_analysis == 'hexagon'):
        local_data = True
    else:
        local_data = False

    ##########################################################################################
    # STEP 1: LOAD DATA
    # ------------------------------ 1.1 LOAD PROXIMITY DATA ------------------------------
    # ------------------------------
    print(f"{area_analysis} - LOAD DATA 1.1 Loading proximity data.")
    # ------------------------------
    # Select unique ID according to selected data
    area_dict = {'unidadesvecinales':'COD_UNICO_',
                'zonascensales':'GEOCODI',  
                'alameda':'name',
                'comunas':'Comuna',
                'hex':'hex_id',
                'santiago':'nom_region'}
    code_column = area_dict[area_analysis]
    
    # Read selected data
    if local_data:
        if area_analysis == 'hexagon':
            prox_gdf = gpd.read_file(grl_dir + f'santiago_hexproximity_{speed_name}_kmh_res{res}.gpkg')
        elif area_analysis == 'alameda':
            prox_gdf = gpd.read_file(grl_dir + f'santiago_alamedaproximity_{speed_name}_kmh.gpkg')
    else:
        if area_analysis == 'hex':
            query = f'SELECT * FROM projects_research.santiago_hexproximity_{speed_name}_kmh WHERE res = {res}'
            prox_gdf = aup.gdf_from_query(query)
        elif area_analysis == 'unidadesvecinales':
            prox_gdf = aup.gdf_from_db(f'santiago_unidadesvecinalesproximity_{speed_name}_kmh','projects_research')
    
        elif area_analysis == 'zonascensales':
            prox_gdf = aup.gdf_from_db(f'santiago_zonascensalesproximity_{speed_name}_kmh','projects_research')
        
        elif area_analysis == 'comunas':
            prox_gdf = aup.gdf_from_db(f'santiago_comunasproximity_{speed_name}_kmh','projects_research')
        
        elif area_analysis == 'santiago':
            prox_gdf = aup.gdf_from_db(f'santiago_{area_analysis}proximity_{speed_name}_kmh','projects_research')
        
    # Change some col names
    prox_gdf = prox_gdf.rename(columns={'paradas_tp_count_15min':'paradas_tp_ruta_count_15min',
                                       'paradas_tp_time':'paradas_tp_ruta_time'})

    # ------------------------------ 1.2 LOAD AREAL DATA ------------------------------
    # ------------------------------
    print(f"{area_analysis} - LOAD DATA 1.2 Loading areal data.")
    # ------------------------------
    if all_local_dirs == 'alex':
        if area_analysis == 'hex':
            hex_dir = f"../../../data/processed/santiago/areal_data/{area_analysis}_areal_res{res}.gpkg"
        else:
            hex_dir = f"../../../data/processed/santiago/areal_data/{area_analysis}_areal.gpkg"
            
    elif all_local_dirs == 'edgar':
        if area_analysis == 'hex':
            hex_dir = f'../../../data/processed/00_pois_formated/aereal_data/{area_analysis}_areal_res{res}.gpkg'
        else:
            hex_dir = f'../../../data/processed/00_pois_formated/aereal_data/{area_analysis}_areal.gpkg'
    
    hex_areal = gpd.read_file(hex_dir)
    hex_areal = hex_areal.rename(columns={'oficinas_sum':'oficinas_count',
                                         'pct_social_viv':'social_viv_count',
                                         'viv_sum':'houses_count',
                                         'pct_hotel':'hotel_count',
                                         'ndvi_mean':'ndvi_count'})
    
    ##########################################################################################
    # STEP 2 : DATA TREATMENT
    # ------------------------------ 2.1 JOIN _priv AND _pub POIS (SOME) ------------------------------
    # ------------------------------
    print(f"{area_analysis} - DATA TREATMENT 2.1 Joining _priv and _pub pois.")
    # ------------------------------
    join_pois_list = ['hospital','clinica','consult_ado', 'museos',
                     'vacunatorio','eq_deportivo',]
    
    for source in join_pois_list:
        # join count columns for private and public in one encompassing column
        prox_gdf[f"{source}_count_15min"] = prox_gdf[f"{source}_priv_count_15min"] + prox_gdf[f"{source}_pub_count_15min"]
        # remove 0 values from time
        prox_gdf.loc[prox_gdf[f"{source}_pub_time"]==0] = np.nan
        prox_gdf.loc[prox_gdf[f"{source}_priv_time"]==0] = np.nan
        # assign general minimum time
        prox_gdf[f"{source}_time"] = prox_gdf[[f"{source}_pub_time", f"{source}_priv_time"]].min(axis=1)
        # remove duplicate info columns
        prox_gdf = prox_gdf.drop(columns=[f"{source}_pub_count_15min", f"{source}_priv_count_15min",
                                         f"{source}_pub_time", f"{source}_priv_time"])
        # fill na with 0 for future processing
        prox_gdf['hospital_time'].fillna(0, inplace=True)

    # ------------------------------ 2.2 MERGE PROXIMITY AND AREAL DATA ------------------------------
    # ------------------------------
    print(f"{area_analysis} - DATA TREATMENT 2.2 Merging proximity and areal data.")
    # ------------------------------
    hex_analysis = hex_areal.merge(prox_gdf.drop(columns='geometry'), on=code_column, how='left')
    hex_analysis = hex_analysis.explode(ignore_index=True)
    hex_analysis = hex_analysis.dissolve(by=code_column)
    hex_analysis = hex_analysis.reset_index()

    ##########################################################################################
    # STEP 3 : HQSL FUNCTION - Step 1/2: Variables analysis
    # ------------------------------ 3.1 USE PREVIOUSLY DEFINED SCALE FUNCTIONS ------------------------------
    # ------------------------------
    print(f"{area_analysis} - HQSL FUNCTION - 3.1 Processing variables analysis.")
    # ------------------------------
    # use scale functions for each column
    for i in tqdm(range(len(weight_dict.keys())),position=0,leave=True):
        # gather specific source
        source = list(weight_dict.keys())[i]
        # iterate over columns
        for col_name in hex_analysis.columns:
            # select column with count information -- refers to the amount of opportunities available at 15 min
            if source in col_name and 'count' in col_name:
                if f'{source}_time' in hex_analysis.columns:
                    hex_analysis[f'{source}_time'].fillna(0, inplace=True)
                hex_analysis[col_name].fillna(0, inplace=True)
                # source scaling
                hex_analysis[f'{source}_scaled'] = hex_analysis[col_name].apply(lambda x:
                                                                                scale_source_fn(
                                                                                    x,
                                                                                source,
                                                                                    weight_dict, 
                                                                    hex_analysis[col_name].mean(),
                                                                    hex_analysis[col_name].std()))
                # treat 0 time values -- hexagons without nodes 
                if area_analysis == 'hex':
                    if weight_dict[source] != 'specific':
                        # assign nan values to hexagons without nodes to avoid affecting the mean calculation process
                        #if source in join_pois_list:
                        #    hex_analysis.loc[hex_analysis.supermercado_time==0,f'{source}_scaled'] = np.nan
                        if source == 'hotel' or source == 'oficinas':
                            continue
                        else:
                            hex_analysis.loc[hex_analysis[f'{source}_time']==0,f'{source}_scaled'] = np.nan
                            
                        # calculate mean count value
                        # print(source)
                        hex_analysis.loc[hex_analysis[f'{source}_time']==0, f'{source}_scaled'] = hex_analysis.loc[hex_analysis[f'{source}_time']==0].apply(lambda x: neighbour_mean(x['hex_id'],
                                                                                'hex_id',
                                                                                hex_analysis,
                                                                                f'{source}_scaled'), axis=1)
    # ------------------------------ 3.2 SAVE ------------------------------
    if upload_preprocessed:
        # ------------------------------
        print(f"{area_analysis} - HQSL FUNCTION - 3.2 Saving variables analysis.")
        # ------------------------------
        hex_preprocessed = hex_analysis.copy()
        if area_analysis == 'hex':
            hex_preprocessed['res'] = res
        hex_preprocessed = hex_preprocessed.dropna()
        table = f'santiago_{area_analysis}variableanalysis_{speed_name}_kmh'
        schema = 'projects_research'
        aup.gdf_to_db_slow(hex_preprocessed, table, schema, if_exists=if_exists)
        del hex_preprocessed

    ##########################################################################################
    # STEP 4 : HQSL FUNCTION - Step 2/2: HQSL Index calculation
    # ------------------------------ 4.1 USE PREVIOUSLY DEFINED INDEX FUNCTIONS ------------------------------
    # ------------------------------
    print(f"{area_analysis} - HQSL FUNCTION - 4.1 Processing HQSL index.")
    # ------------------------------
    hex_ind = indicator_fn(hex_analysis, parameters_dict)
    hex_social_fn = social_fn(hex_analysis, parameters_dict)
    hex_hqsl = hqsl_fn(hex_social_fn, parameters_dict)
    
    hex_idx = hex_ind.merge(hex_social_fn.drop(columns='geometry'), on=code_column)
    hex_idx = hex_idx.merge(hex_hqsl.drop(columns='geometry'), on=code_column)

    # ------------------------------ 4.2 SAVE ------------------------------
    if area_analysis == 'hex':
        hex_idx['res'] = res
    
    hex_idx = hex_idx.dropna()
    
    if save:
        # ------------------------------
        print(f"{area_analysis} - HQSL FUNCTION - 4.2 Saving HQSL index to database.")
        # ------------------------------
        table = f'santiago_{area_analysis}analysis_{speed_name}_kmh'
        schema = 'projects_research'
        aup.gdf_to_db_slow(hex_idx, table, schema, if_exists=if_exists)
    if save_local:
        # ------------------------------
        print(f"{area_analysis} - HQSL FUNCTION - 4.2 Saving HQSL index locally.")
        # ------------------------------
        hex_idx.to_file(grl_dir + f'santiago_{area_analysis}analysis_{speed_name}_kmh.geojson')

--------------------------------------------------------------------------------
--- RUNNING CALCULATION FOR AREA OF ANALYSIS: santiago.
santiago - LOAD DATA 1.1 Loading proximity data.
santiago - LOAD DATA 1.2 Loading areal data.
santiago - DATA TREATMENT 2.1 Joining _priv and _pub pois.
santiago - DATA TREATMENT 2.2 Merging proximity and areal data.
santiago - HQSL FUNCTION - 3.1 Processing variables analysis.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 53/53 [00:00<00:00, 1478.70it/s]

santiago - HQSL FUNCTION - 3.2 Saving variables analysis.


santiago - HQSL FUNCTION - 4.1 Processing HQSL index.
santiago - HQSL FUNCTION - 4.2 Saving HQSL index to database.
--------------------------------------------------------------------------------
--- RUNNING CALCULATION FOR AREA OF ANALYSIS: comunas.
comunas - LOAD DATA 1.1 Loading proximity data.
comunas - LOAD DATA 1.2 Loading areal data.
comunas - DATA TREATMENT 2.1 Joining _priv and _pub pois.
comunas - DATA TREATMENT 2.2 Merging proximity and areal data.
comunas - HQSL FUNCTION - 3.1 Processing variables analysis.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 53/53 [00:00<00:00, 388.30it/s]

...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 53/53 [00:03<00:00, 17.57it/s]


unidadesvecinales - HQSL FUNCTION - 3.2 Saving variables analysis.
unidadesvecinales - HQSL FUNCTION - 4.1 Processing HQSL index.
unidadesvecinales - HQSL FUNCTION - 4.2 Saving HQSL index to database.
--------------------------------------------------------------------------------
--- RUNNING CALCULATION FOR AREA OF ANALYSIS: alameda.
alameda - LOAD DATA 1.1 Loading proximity data.
alameda - LOAD DATA 1.2 Loading areal data.
alameda - DATA TREATMENT 2.1 Joining _priv and _pub pois.
alameda - DATA TREATMENT 2.2 Merging proximity and areal data.
alameda - HQSL FUNCTION - 3.1 Processing variables analysis.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 53/53 [00:00<00:00, 1066.90it/s]

alameda - HQSL FUNCTION - 3.2 Saving variables analysis.


alameda - HQSL FUNCTION - 4.1 Processing HQSL index.
alameda - HQSL FUNCTION - 4.2 Saving HQSL index to database.
